# ◎ 한국도로공사 - 고속도로 공공데이터 포털 
---


## 1. 데이터 수집
### 1) 특송기간 교통분석 - 오픈api | [url](http://data.ex.co.kr/openapi/intro/introduce01)
* 특송기간(연말연시, 설날, 추석) 도시간 소요시간 및 시간대별 교통량 현황 데이터   
   
<img src="./img/img1.PNG" width="500" height="800">

* 1. 도시간 소요시간 - 연말연시, 설, 추석 기준으로 출발지점에서 도착지점까지 소요시간 데이터
    * 17년도 이전 데이터는 없음
    * 기본정보
        * url : 'http://data.ex.co.kr/openapi/specialAnal/intercityLeadTime'
        * 호출방법 : HTTP GET, HTTP POST
        * 결과포맷 : XML, JSON
    * 요청변수
        * key(필수)   : string | 발급받은 인증키
        * type(필수)  : string | 검색결과 포맷
        * iYear(필수) : string | 기준년도
        * iStUnitCode : string | 출발영업소코드
        * iEdUnitCode : string | 도착영업소코드
        * ~~numOfRows   : string | 한 페이지당 출력건수~~
        * ~~pageNo      : string | 출력 페이지번호~~
    * 출력결과
        * stdYear              : string | 기준년도
        * sphlDfttNm	       : string	| 특수일구분명
        * ~~sphlDfttCode	       : string	| 특수일구분코드~~
        * sphlDfttScopTypeNm   : string	| 특수일전후특송기간범위구분명
        * ~~sphlDfttScopTypeCode : string	| 특수일전후특송기간범위구분코드~~
        * stdHour	           : string	| 기준시
        * stUnitNm	           : string	| 출발영업소명
        * stUnitCode           : string	| 출발영업소코드
        * edUnitNm	           : string	| 도착영업소명
        * edUnitCode	       : string	| 도착영업소코드
        * dirNm	               : string	| 방향명
        * ~~dirCode              : string	| 방향코드~~
        * trtm	               : string	| 통행시간(분)
        * ~~numOfRows	           : string	| 한 페이지당 출력건수~~
        * ~~pageNo	           : string	| 출력 페이지번호~~
    * 영업소코드 
        * 직접 찾아봐야 할 듯
        *  ex) 서울 - 101 | 부산 - 140 | ...
    * 출력결과 사진   
       
<img src="./img/result1.PNG" width="500" height="600">

* 2. 시간대별 교통량 현황 
    * 정렬이 안되어있음
    * 기본정보
        * url : 'http://data.ex.co.kr/openapi/specialAnal/trafficFlowByTime'
        * 호출방법 : HTTP GET, HTTP POST
        * 결과포맷 : XML, JSON
    * 요청변수
        * key	   : string	| 발급받은 인증키
        * type	   : string	| 검색결과 포맷
        * iStdYear : string	| 기준년
    * 출력결과
        * stdYear	           : string | 기준년
        * sphlDfttNm	       : string	| 특수일구분명
        * ~~sphlDfttCode	       : string	| 특수일구분코드~~
        * sphlDfttScopTypeNm   : string | 특수일전후특송기간범위구분명
        * ~~sphlDfttScopTypeCode : string | 특수일전후특송기간범위구분코드~~
        * stdHour	           : string	| 기준시
        * trfl                 : string | 교통량(대)   
           
<img src="./img/result2.PNG" width="500" height="500">

* 1,2번 데이터 모두 매 시간 다 있는게 아니라 중간에 빠진 데이터가 있으니 유의하시길...


In [1]:
import pandas as pd
import requests, json

def timeRequired(Year, totalPage):
    time = pd.DataFrame(columns = ['기준년도', '특수일구분명', '특수일전후특송기간범위구분명', '특수일전후특송기간범위구분코드', '기준시', '통행시간(분)'])
    
    idx = 0
    
    for pageNo in range(1, totalPage+1):
        url = 'http://data.ex.co.kr/openapi/specialAnal/intercityLeadTime'
        # input your key code in 'key'
        parameters = { 'key' : 'your key', 'type' : 'json', 'iYear' : str(Year), 'iStUnitCode' : '101', 
                        'iEdUnitCode' : '140', 'numOfRows' : '10', 'pageNo' : str(pageNo)}
        response = requests.get(url, params=parameters)
        informations = response.json()['intercityLeadTimeLists']

        for info in informations : 
            if((info['sphlDfttNm'] == '설날') or (info['sphlDfttNm'] == '추석')):
                time.loc[idx] = [Year, info['sphlDfttNm'], info['sphlDfttScopTypeNm'], info['sphlDfttScopTypeCode'], info['stdHour'], info['trtm']]
                idx += 1

    time.to_csv(f'./rawdata/time_{Year}.csv', sep=',', encoding='utf-8', index=False)
    
    return time

def trafficVolume(Year):
    traffic = pd.DataFrame(columns=['기준년도', '특수일구분명', '특수일전후특송기간범위구분명', '특수일전후특송기간범위구분코드', '기준시', '교통량(대)'])

    url = 'http://data.ex.co.kr/openapi/specialAnal/trafficFlowByTime'
    # input your key code in 'key'
    parameters = {'key' : 'your key', 'type' : 'json' , 'iStdYear' : str(Year)}
    response = requests.get(url, params=parameters)
    informations = response.json()['trafficFlowByTimeLists']
    
    idx = 0
    for info in informations : 
        if((info['sphlDfttNm'] == '설날') or (info['sphlDfttNm'] == '추석')):
            traffic.loc[idx] = [Year, info['sphlDfttNm'], info['sphlDfttScopTypeNm'], info['sphlDfttScopTypeCode'], info['stdHour'], info['trfl']]
            idx += 1
    
    traffic.to_csv(f'./rawdata/traffic_{Year}.csv', sep=',', encoding='utf-8', index=False)
    
    return traffic

**Time Consuming Code (about 2~3min)** - timeRequired()

In [2]:
# 1. 도시간 소요시간
time_21 = timeRequired(2021, 42)

# 2. 시간대별 교통량 현황
traffic_21 = trafficVolume(2021)

In [4]:
time_21 = pd.read_csv('./rawdata/time_2021.csv', sep=',', encoding='utf-8')
traf_21 = pd.read_csv('./rawdata/traffic_2021.csv', sep=',', encoding='utf-8')

In [5]:
time_21

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,통행시간(분)
0,2021,설날,D-3,1,0,249.050
1,2021,설날,D-3,1,1,249.100
2,2021,설날,D-3,1,2,249.150
3,2021,설날,D-3,1,3,NaN
4,2021,설날,D-3,1,4,NaN
...,...,...,...,...,...,...
322,2021,추석,D+3,7,18,277.775
323,2021,추석,D+3,7,19,286.816
324,2021,추석,D+3,7,20,301.116
325,2021,추석,D+3,7,21,315.433


In [6]:
traffic_21

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,교통량(대)
0,2021,설날,D+3,7,08,296016
1,2021,설날,D-2,2,13,288936
2,2021,추석,D-1,3,21,150673
3,2021,추석,D+3,7,15,302304
4,2021,추석,D,4,18,284840
...,...,...,...,...,...,...
331,2021,설날,D,4,14,362386
332,2021,추석,D-1,3,05,53276
333,2021,설날,D+1,5,14,331326
334,2021,추석,D+2,6,05,115809


In [7]:
time_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준년도             327 non-null    int64  
 1   특수일구분명           327 non-null    object 
 2   특수일전후특송기간범위구분명   327 non-null    object 
 3   특수일전후특송기간범위구분코드  327 non-null    int64  
 4   기준시              327 non-null    int64  
 5   통행시간(분)          323 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 15.5+ KB


In [8]:
traffic_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   기준년도             336 non-null    object
 1   특수일구분명           336 non-null    object
 2   특수일전후특송기간범위구분명   336 non-null    object
 3   특수일전후특송기간범위구분코드  336 non-null    object
 4   기준시              336 non-null    object
 5   교통량(대)           336 non-null    object
dtypes: object(6)
memory usage: 18.4+ KB


In [9]:
traffic_21['기준년도']=traffic_21['기준년도'].astype('int')
traffic_21['특수일전후특송기간범위구분코드']=traffic_21['특수일전후특송기간범위구분코드'].astype(int)
traffic_21['기준시']=traffic_21['기준시'].astype('int')

In [10]:
traffic_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   기준년도             336 non-null    int32 
 1   특수일구분명           336 non-null    object
 2   특수일전후특송기간범위구분명   336 non-null    object
 3   특수일전후특송기간범위구분코드  336 non-null    int32 
 4   기준시              336 non-null    int32 
 5   교통량(대)           336 non-null    object
dtypes: int32(3), object(3)
memory usage: 14.4+ KB


In [11]:
time_21.isna().sum()

기준년도               0
특수일구분명             0
특수일전후특송기간범위구분명     0
특수일전후특송기간범위구분코드    0
기준시                0
통행시간(분)            4
dtype: int64

In [12]:
traffic_21.isna().sum()

기준년도               0
특수일구분명             0
특수일전후특송기간범위구분명     0
특수일전후특송기간범위구분코드    0
기준시                0
교통량(대)             0
dtype: int64

In [13]:
time_21[time_21['통행시간(분)'].isna()==True]

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,통행시간(분)
3,2021,설날,D-3,1,3,NaN
4,2021,설날,D-3,1,4,NaN
122,2021,설날,D+2,6,2,NaN
286,2021,추석,D+2,6,5,NaN


In [14]:
# time 결측치 interpolate로 정리
time_21['통행시간(분)']=time_21['통행시간(분)'].interpolate()
time_21

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,통행시간(분)
0,2021,설날,D-3,1,0,249.050000
1,2021,설날,D-3,1,1,249.100000
2,2021,설날,D-3,1,2,249.150000
3,2021,설날,D-3,1,3,263.455333
4,2021,설날,D-3,1,4,277.760667
...,...,...,...,...,...,...
322,2021,추석,D+3,7,18,277.775000
323,2021,추석,D+3,7,19,286.816000
324,2021,추석,D+3,7,20,301.116000
325,2021,추석,D+3,7,21,315.433000


In [15]:
time_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준년도             327 non-null    int64  
 1   특수일구분명           327 non-null    object 
 2   특수일전후특송기간범위구분명   327 non-null    object 
 3   특수일전후특송기간범위구분코드  327 non-null    int64  
 4   기준시              327 non-null    int64  
 5   통행시간(분)          327 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 15.5+ KB


In [16]:
traffic_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   기준년도             336 non-null    int32 
 1   특수일구분명           336 non-null    object
 2   특수일전후특송기간범위구분명   336 non-null    object
 3   특수일전후특송기간범위구분코드  336 non-null    int32 
 4   기준시              336 non-null    int32 
 5   교통량(대)           336 non-null    object
dtypes: int32(3), object(3)
memory usage: 14.4+ KB


In [17]:
# 정렬 및 인덱스 정리
time_21.sort_values(by=['기준년도','특수일구분명','특수일전후특송기간범위구분코드', '기준시'], ascending=[True,True,True,True], inplace=True)
traffic_21.sort_values(by=['기준년도','특수일구분명','특수일전후특송기간범위구분코드', '기준시'], ascending=[True,True,True,True], inplace=True)
time_21.reset_index(drop=True, inplace=True)
traffic_21.reset_index(drop=True, inplace=True)

In [18]:
time_21

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,통행시간(분)
0,2021,설날,D-3,1,0,249.050000
1,2021,설날,D-3,1,1,249.100000
2,2021,설날,D-3,1,2,249.150000
3,2021,설날,D-3,1,3,263.455333
4,2021,설날,D-3,1,4,277.760667
...,...,...,...,...,...,...
322,2021,추석,D+3,7,18,277.775000
323,2021,추석,D+3,7,19,286.816000
324,2021,추석,D+3,7,20,301.116000
325,2021,추석,D+3,7,21,315.433000


In [19]:
traffic_21

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,교통량(대)
0,2021,설날,D-3,1,0,34122
1,2021,설날,D-3,1,1,24920
2,2021,설날,D-3,1,2,21026
3,2021,설날,D-3,1,3,21582
4,2021,설날,D-3,1,4,30840
...,...,...,...,...,...,...
331,2021,추석,D+3,7,19,260166
332,2021,추석,D+3,7,20,200270
333,2021,추석,D+3,7,21,158047
334,2021,추석,D+3,7,22,118968


In [20]:
pd.merge(time_21, traffic_21, on=['기준년도','특수일구분명','특수일전후특송기간범위구분명','특수일전후특송기간범위구분코드','기준시'], how='left')

,기준년도,특수일구분명,특수일전후특송기간범위구분명,특수일전후특송기간범위구분코드,기준시,통행시간(분),교통량(대)
0,2021,설날,D-3,1,0,249.050000,34122
1,2021,설날,D-3,1,1,249.100000,24920
2,2021,설날,D-3,1,2,249.150000,21026
3,2021,설날,D-3,1,3,263.455333,21582
4,2021,설날,D-3,1,4,277.760667,30840
...,...,...,...,...,...,...,...
322,2021,추석,D+3,7,18,277.775000,322776
323,2021,추석,D+3,7,19,286.816000,260166
324,2021,추석,D+3,7,20,301.116000,200270
325,2021,추석,D+3,7,21,315.433000,158047
